In [3]:
!pip install matminer seaborn
from matminer.datasets import load_dataset
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 22.6 MB/s eta 0:00:00


In [1]:
!git clone https://github.com/txie-93/cgcnn.git
%cd cgcnn
!pip install numpy pandas scikit-learn pymatgen tqdm
!pip install torch==1.13.1

Cloning into 'cgcnn'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 127 (delta 27), reused 27 (delta 27), pack-reused 90 (from 1)
Receiving objects: 100% (127/127), 7.23 MiB | 14.37 MiB/s, done.
Resolving deltas: 100% (59/59), done.
/content/cgcnn
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.0/809.0 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/7

In [15]:
from matminer.datasets import load_dataset
from pymatgen.core import Structure
import pandas as pd
import os

df = load_dataset("castelli_perovskites")
df = df[df["gap gllbsc"].notnull()].reset_index(drop=True)

# ディレクトリ作成
os.makedirs("raw", exist_ok=True)

records = []
for idx, row in df.iterrows():
    structure: Structure = row["structure"]
    cif_filename = f"{idx}.cif"
    structure.to(fmt="cif", filename=os.path.join("raw", cif_filename))
    records.append([cif_filename, row["gap gllbsc"]])

df_label = pd.DataFrame(records, columns=["id", "gap"])
df_label.to_csv("id_prop.csv", index=False)


In [16]:
import os
assert os.path.exists("id_prop.csv"), "❌ id_prop.csv が存在しません！"
assert os.path.exists("raw"), "❌ raw/ ディレクトリが存在しません！"
print("✅ 必須ファイルが確認できました！")

✅ 必須ファイルが確認できました！


In [17]:
!python utils/atom_init.py raw id_prop.csv atom_init.json

python3: can't open file '/content/cgcnn/utils/atom_init.py': [Errno 2] No such file or directory


In [21]:
import json
from pymatgen.core import Element

atom_init = {}
for Z in range(1, 119):
    el = Element.from_Z(Z)
    atom_init[str(Z)] = [
        el.atomic_mass,
        el.mendeleev_no,
        el.atomic_radius,
        getattr(el, "X", 0.0),  # ← electronegativityの代替
        el.row,
        el.group or 0,
        el.block == "s",
        el.block == "p",
        el.block == "d",
        el.block == "f",
    ]

with open("/content/atom_init.json", "w") as f:
    json.dump(atom_init, f)

print("✅ atom_init.json を生成しました！")


✅ atom_init.json を生成しました！


<ipython-input-21-1eec336f772b>:11: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  getattr(el, "X", 0.0),  # ← electronegativityの代替
<ipython-input-21-1eec336f772b>:11: UserWarning: No Pauling electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  getattr(el, "X", 0.0),  # ← electronegativityの代替
<ipython-input-21-1eec336f772b>:11: UserWarning: No Pauling electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  getattr(el, "X", 0.0),  # ← electronegativityの代替
<ipython-input-21-1eec336f772b>:9: UserWarning: No data available for mendeleev_no for Rf
  el.mendeleev_no,
<ipython-input-21-1eec336f772b>:11: UserWarning: No Pauling electronegativity for Rf. Setting to NaN. This has no physical 

In [22]:
from matminer.datasets import load_dataset
from pymatgen.core import Structure
import pandas as pd
import os

df = load_dataset("castelli_perovskites")
df = df[df["gap gllbsc"].notnull()].reset_index(drop=True)

os.makedirs("/content/cgcnn/raw", exist_ok=True)

records = []
for idx, row in df.iterrows():
    structure: Structure = row["structure"]
    cif_filename = f"{idx}.cif"
    structure.to(fmt="cif", filename=os.path.join("/content/cgcnn/raw", cif_filename))
    records.append([cif_filename, row["gap gllbsc"]])

df_label = pd.DataFrame(records, columns=["id", "gap"])
df_label.to_csv("/content/cgcnn/id_prop.csv", index=False)

print("✅ id_prop.csv と raw/*.cif を /content/cgcnn に準備完了！")


✅ id_prop.csv と raw/*.cif を /content/cgcnn に準備完了！


In [41]:
config = {
  "task": "regression",
  "atom_init_path": "atom_init.json",
  "id_prop_path": "id_prop.csv",
  "train_ratio": 0.8,
  "val_ratio": 0.1,
  "test_ratio": 0.1,
  "atom_fea_len": 64,
  "n_conv": 3,
  "h_fea_len": 128,
  "n_h": 1,
  "optim": "SGD",
  "lr": 0.01,
  "momentum": 0.9,
  "weight_decay": 0.,
  "batch_size": 32,
  "epochs": 30,
  "print_freq": 10,
  "num_workers": 0,
  "data_path": "raw"
}


import json
with open("config.json", "w") as f:
    json.dump(config, f, indent=4)

print("✅ config.json を作成しました")


✅ config.json を作成しました


In [25]:
!python main.py config.json raw

Traceback (most recent call last):
  File "/content/cgcnn/main.py", line 513, in <module>
    main()
  File "/content/cgcnn/main.py", line 95, in main
    dataset = CIFData(*args.data_options)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/cgcnn/cgcnn/data.py", line 306, in __init__
    assert os.path.exists(id_prop_file), 'id_prop.csv does not exist!'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: id_prop.csv does not exist!


In [26]:
%cd /content/cgcnn

!mkdir -p data/raw
!mv *.cif data/raw/
!mv atom_init.json data/
!mv id_prop.csv data/


/content/cgcnn
mv: cannot stat '*.cif': No such file or directory
mv: cannot stat 'atom_init.json': No such file or directory


In [27]:
import os

print("📂 ファイル構成チェック：")
for root, dirs, files in os.walk("/content/cgcnn/data"):
    level = root.replace("/content/cgcnn", "").count(os.sep)
    indent = "  " * level
    print(f"{indent}{os.path.basename(root)}/")
    for f in files:
        print(f"{indent}  {f}")


📂 ファイル構成チェック：
  data/
    id_prop.csv
    id_prop/
      gap.csv
    sample-classification/
      9012304.cif
      atom_init.json
      9009743.cif
      9011998.cif
      9011050.cif
      1000041.cif
      1507756.cif
      9000046.cif
      id_prop.csv
      7206075.cif
      1000050.cif
      1101051.cif
    material-data/
      mp-ids-27430.csv
      mp-ids-3402.csv
      mp-ids-46744.csv
      README.md
    raw/
      8810.cif
      7661.cif
      17256.cif
      11784.cif
      17626.cif
      11650.cif
      10900.cif
      12483.cif
      7982.cif
      13346.cif
      4781.cif
      563.cif
      586.cif
      5313.cif
      6256.cif
      4594.cif
      12215.cif
      9305.cif
      149.cif
      16382.cif
      8093.cif
      9796.cif
      8244.cif
      7171.cif
      6061.cif
      15924.cif
      10706.cif
      17404.cif
      8359.cif
      159.cif
      5119.cif
      8885.cif
      6553.cif
      13969.cif
      18597.cif
      17493.cif
      16756.cif
      1301

In [28]:
!python main.py config.json data


Traceback (most recent call last):
  File "/content/cgcnn/main.py", line 513, in <module>
    main()
  File "/content/cgcnn/main.py", line 95, in main
    dataset = CIFData(*args.data_options)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/cgcnn/cgcnn/data.py", line 306, in __init__
    assert os.path.exists(id_prop_file), 'id_prop.csv does not exist!'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: id_prop.csv does not exist!


In [29]:
!find /content -name "*.cif" | head
!find /content -name "atom_init.json"
!find /content -name "id_prop.csv"


/content/cgcnn/raw/0.cif
/content/cgcnn/raw/1.cif
/content/cgcnn/raw/2.cif
/content/cgcnn/raw/3.cif
/content/cgcnn/raw/4.cif
/content/cgcnn/raw/5.cif
/content/cgcnn/raw/6.cif
/content/cgcnn/raw/7.cif
/content/cgcnn/raw/8.cif
/content/cgcnn/raw/9.cif
/content/atom_init.json
/content/cgcnn/data/sample-classification/atom_init.json
/content/cgcnn/data/sample-regression/atom_init.json
/content/cgcnn/data/sample-classification/id_prop.csv
/content/cgcnn/data/id_prop.csv
/content/cgcnn/data/sample-regression/id_prop.csv


In [30]:
!cp /content/atom_init.json /content/cgcnn/data/
!mkdir -p /content/cgcnn/data/raw
!cp /content/cgcnn/raw/*.cif /content/cgcnn/data/raw/


In [31]:
%cd /content/cgcnn

!python main.py config.json data


/content/cgcnn
Traceback (most recent call last):
  File "/content/cgcnn/main.py", line 513, in <module>
    main()
  File "/content/cgcnn/main.py", line 95, in main
    dataset = CIFData(*args.data_options)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/cgcnn/cgcnn/data.py", line 306, in __init__
    assert os.path.exists(id_prop_file), 'id_prop.csv does not exist!'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: id_prop.csv does not exist!


In [32]:
!ls /content/cgcnn/data/id_prop.csv


/content/cgcnn/data/id_prop.csv


In [33]:
# カレントディレクトリにコピー（main.pyが探しやすくする）
!cp /content/cgcnn/data/id_prop.csv /content/cgcnn/id_prop.csv
!cp /content/cgcnn/data/atom_init.json /content/cgcnn/atom_init.json
!mkdir -p /content/cgcnn/raw
!cp /content/cgcnn/data/raw/*.cif /content/cgcnn/raw/


In [34]:
%cd /content/cgcnn

!python main.py config.json raw


/content/cgcnn
Traceback (most recent call last):
  File "/content/cgcnn/main.py", line 513, in <module>
    main()
  File "/content/cgcnn/main.py", line 95, in main
    dataset = CIFData(*args.data_options)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/cgcnn/cgcnn/data.py", line 306, in __init__
    assert os.path.exists(id_prop_file), 'id_prop.csv does not exist!'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: id_prop.csv does not exist!


In [35]:
!ls -R /content/cgcnn


ストリーミング出力は最後の 5000 行に切り捨てられました。
10383.cif  12817.cif  15250.cif  17685.cif  3082.cif  5516.cif	7950.cif
10384.cif  12818.cif  15251.cif  17686.cif  3083.cif  5517.cif	7951.cif
10385.cif  12819.cif  15252.cif  17687.cif  3084.cif  5518.cif	7952.cif
10386.cif  1281.cif   15253.cif  17688.cif  3085.cif  5519.cif	7953.cif
10387.cif  12820.cif  15254.cif  17689.cif  3086.cif  551.cif	7954.cif
10388.cif  12821.cif  15255.cif  1768.cif   3087.cif  5520.cif	7955.cif
10389.cif  12822.cif  15256.cif  17690.cif  3088.cif  5521.cif	7956.cif
1038.cif   12823.cif  15257.cif  17691.cif  3089.cif  5522.cif	7957.cif
10390.cif  12824.cif  15258.cif  17692.cif  308.cif   5523.cif	7958.cif
10391.cif  12825.cif  15259.cif  17693.cif  3090.cif  5524.cif	7959.cif
10392.cif  12826.cif  1525.cif	 17694.cif  3091.cif  5525.cif	795.cif
10393.cif  12827.cif  15260.cif  17695.cif  3092.cif  5526.cif	7960.cif
10394.cif  12828.cif  15261.cif  17696.cif  3093.cif  5527.cif	7961.cif
10395.cif  12829.cif  15262.cif  17

In [36]:
# 1. カレントディレクトリに必要ファイルを移動
!cp /content/cgcnn/data/id_prop.csv /content/cgcnn/id_prop.csv
!cp /content/cgcnn/data/atom_init.json /content/cgcnn/atom_init.json


In [37]:
%cd /content/cgcnn

!python main.py config.json raw


/content/cgcnn
Traceback (most recent call last):
  File "/content/cgcnn/main.py", line 513, in <module>
    main()
  File "/content/cgcnn/main.py", line 95, in main
    dataset = CIFData(*args.data_options)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/cgcnn/cgcnn/data.py", line 306, in __init__
    assert os.path.exists(id_prop_file), 'id_prop.csv does not exist!'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: id_prop.csv does not exist!


In [38]:
!ls -l /content/cgcnn


total 844
-rw-r--r-- 1 root root   7922 May 22 06:38 atom_init.json
drwxr-xr-x 3 root root   4096 May 22 06:26 cgcnn
-rw-r--r-- 1 root root    415 May 22 06:34 config.json
drwxr-xr-x 7 root root   4096 May 22 06:36 data
-rw-r--r-- 1 root root 253890 May 22 06:38 id_prop.csv
-rw-r--r-- 1 root root   1065 May 22 06:22 LICENSE
-rw-r--r-- 1 root root  20707 May 22 06:22 main.py
-rw-r--r-- 1 root root  11219 May 22 06:22 predict.py
drwxr-xr-x 2 root root   4096 May 22 06:22 pre-trained
drwxr-xr-x 2 root root 528384 May 22 06:26 raw
-rw-r--r-- 1 root root   8225 May 22 06:22 README.md


In [45]:
!python main.py config.json raw


Traceback (most recent call last):
  File "/content/cgcnn/main.py", line 513, in <module>
    main()
  File "/content/cgcnn/main.py", line 95, in main
    dataset = CIFData(*args.data_options)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/cgcnn/cgcnn/data.py", line 306, in __init__
    assert os.path.exists(id_prop_file), 'id_prop.csv does not exist!'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: id_prop.csv does not exist!
